In [ ]:
%pip install requests
%pip install pandas
%pip install plotly
%pip install osmnx
%pip install folium
%pip install scipy scikit-learn

In [1]:
import requests as rq
import os
import json
import pandas as pd
import IPython
import plotly.express as px
import osmnx as ox
import folium
import geopandas as gpd

DATA_BASE = "data"
TIMETABLE_FILE = "timetable"
STOPS_FILE = "stops"
PATHS_FILE = "paths"
FILE_EXT = ".json"

In [2]:
ROUTE_API = lambda id: f"http://apicms.ebms.vn/businfo/getroutebyid/{id}"
TIMETABLE_API = lambda route: f"http://apicms.ebms.vn/businfo/gettimetablebyroute/{route}"
ROUTEVAR_API = lambda route: f"http://apicms.ebms.vn/businfo/getvarsbyroute/{route}"
STOPS_API = lambda id, varId: f"http://apicms.ebms.vn/businfo/getstopsbyvar/{id}/{varId}"
PATHS_API = lambda id, varId: f"http://apicms.ebms.vn/businfo/getpathsbyvar/{id}/{varId}"

def LoadData(route, source, api):
    source += FILE_EXT
    dir_path = os.path.join(os.getcwd(), DATA_BASE, str(route))
    file_path = os.path.join(os.getcwd(), DATA_BASE, str(route), source)

    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

    if not os.path.exists(file_path):
        response = rq.get(api)
        with open(file_path, "w+", encoding="utf-8") as f:
            string = json.dumps(response.json(), ensure_ascii=False)
            f.write(string)
            
    return pd.read_json(file_path, encoding="utf-8")

In [3]:
class StationModel():
    def __init__(self, routeId, varId) -> None:
        src = STOPS_FILE + str(varId)
        api = STOPS_API(id=routeId, varId=varId)
        stops_df = LoadData(route=routeId, source=src, api=api)
        self.stops_df = stops_df[["StopId", "Lat", "Lng"]]

In [4]:
class BusModel():
    def __init__(self, id) -> None:
        self.stations = dict()
        self.paths_df = dict()
        timetables_df = LoadData(route=id, source=TIMETABLE_FILE, api=TIMETABLE_API(id))
        for varId in timetables_df["RouteVarId"]:
            self.stations[varId] = StationModel(routeId=id, varId=varId)
            self.paths_df[varId] = LoadData(route=id, source=PATHS_FILE, api=PATHS_API(id, varId=varId))

In [75]:
buses = list()
bus_ids = [8]
for id in bus_ids:
    buses.append(BusModel(id))

In [6]:
HCM_NETWORK = "./data/hcm.gpkg"
if not os.path.exists(HCM_NETWORK):
    G = ox.graph_from_place("Ho Chi Minh City", simplify=False)
    ox.save_graph_geopackage(G, HCM_NETWORK, directed=True)
    
gdf_nodes = gpd.read_file(HCM_NETWORK, layer="nodes").set_index('osmid')
gdf_edges = gpd.read_file(HCM_NETWORK, layer="edges").set_index(['u', 'v', 'key'])
graph_attrs = {'crs': 'epsg:4326', 'simplified': True}
G = ox.graph_from_gdfs(gdf_nodes=gdf_nodes, gdf_edges=gdf_edges, graph_attrs=graph_attrs)

# fig, ax = ox.plot_graph(G, node_size=0)

In [7]:
HCM_NETWORK = "./data/hcm.osm"
utn = ox.settings.useful_tags_node
oxna = ox.settings.osm_xml_node_attrs
oxnt = ox.settings.osm_xml_node_tags
utw = ox.settings.useful_tags_way
oxwa = ox.settings.osm_xml_way_attrs
oxwt = ox.settings.osm_xml_way_tags
utn = list(set(utn + oxna + oxnt))
utw = list(set(utw + oxwa + oxwt))
ox.settings.all_oneway = True
ox.settings.useful_tags_node = utn
ox.settings.useful_tags_way = utw
if not os.path.exists(HCM_NETWORK):
    G = ox.graph_from_place("Ho Chi Minh City")
    ox.save_graph_xml(G, HCM_NETWORK)
    
G = ox.graph_from_xml(HCM_NETWORK)

# fig, ax = ox.plot_graph(G, node_size=0)

In [8]:
ox.distance.add_edge_lengths(G)
ox.add_edge_speeds(G)
ox.add_edge_travel_times(G)

In [302]:
paths = buses[0].paths_df[15]
paths["nodes"] = ox.nearest_nodes(G, X=paths["lng"], Y=paths["lat"])
# paths = paths.drop_duplicates(subset=["nodes"]).reset_index().drop("index", axis="columns")
# paths["edges"] = ox.nearest_edges(G, X=paths["lng"], Y=paths["lat"])
# paths = paths.drop_duplicates(subset=["edges"]).reset_index().drop("index", axis="columns")
paths

,lat,lng,nodes,edges
0,10.733545,106.656357,2495870411,"(5755051493, 5755051489, 0)"
1,10.733545,106.656357,2495870411,"(5755051493, 5755051489, 0)"
2,10.733556,106.656685,6432690948,"(5755051493, 5755051489, 0)"
3,10.734046,106.656685,5755051489,"(4652534427, 5755051489, 0)"
4,10.734031,106.656166,4652534427,"(2495870411, 4652534427, 0)"
...,...,...,...,...
609,10.876465,106.801552,3342113628,"(3342113628, 3342113621, 0)"
610,10.875196,106.801453,3342113628,"(3342113628, 3342113621, 0)"
611,10.874263,106.801407,3342113628,"(3342113628, 3342113621, 0)"
612,10.874231,106.802055,3342113628,"(3342113628, 3342113621, 0)"


In [312]:
def find_route(path):
    next_row = path.name + 1 if path.name + 1 < paths.shape[0] else path.name
    node_ids = ox.shortest_path(G, path["nodes"], paths.iloc[next_row]["nodes"])
    return node_ids
routes = paths.apply(find_route, axis="columns", result_type='reduce').drop_duplicates().dropna()
routes

0      [2495870411, 2495870411]
1      [2495870411, 6432690948]
2      [6432690948, 5755051489]
3      [5755051489, 4652534427]
4      [4652534427, 5755051488]
                 ...           
591    [5762890408, 3342113619]
592    [3342113619, 3342113619]
595    [3342113619, 3342113621]
596    [3342113621, 3342113628]
597    [3342113628, 3342113628]
Length: 423, dtype: object

In [268]:
paths = buses[0].stations[15].stops_df
paths["nodes"] = ox.nearest_nodes(G, X=paths["Lng"], Y=paths["Lat"])
def find_route(path):
    next_row = path.name + 1 if path.name + 1 < paths.shape[0] else path.name
    node_ids = ox.shortest_path(G, path["nodes"], paths.iloc[next_row]["nodes"])
    return node_ids
routes = paths.apply(find_route, axis="columns", result_type='reduce')
routes = routes.dropna().reset_index().drop("index", axis="columns")[0]
routes

0     [2495870411.0, 4652534427, 5755051488, 3664505...
1     [6432751391.0, 5755153782, 6758003387, 2040317...
2     [6449960735.0, 3116882965, 6757924464, 6449960...
3     [6450096946.0, 4616320836, 366442410, 67703910...
4     [6727805054.0, 5816921541, 6770923492, 6783111...
                            ...                        
56    [5778173023.0, 1958801546, 5778173032, 6778322...
57    [5778218361.0, 5778390214, 6616505160, 5778390...
58    [4831043101.0, 5762890412, 4831043103, 4831043...
59                                       [3342113628.0]
60                                       [3342113628.0]
Name: 0, Length: 61, dtype: object

In [313]:
route_map = None
for route in routes:
    try:
        route_map = ox.plot_route_folium(G, route, route_map, node_size=0, show=False, tiles="openstreetmap")
    except:
        # display(route)
        continue

In [314]:
route_map